# Create an Index with custom cracking and chunking (in Parallel) using Azure Document Intelligence

Create an index with custom cracking and chunking using the Azure Document Intelligence aka Azure Form Recognizer.

This notebook does crack_and_chunk with parallel processing. It runs on a compute cluster with multiple nodes and each has multiple processes, the input files are divided into mini batches and assigned to each process. All processes do the crack_and_chunk in parallel.

This version works with generate_embeddings serial component, with set_automatic_compoute() function.

In [1]:
%pip install -U azure-ai-ml==1.16.0b1  # versions may change later
%pip install azure-identity
%pip install -U 'azureml-rag[azure,cognitive_search]>=0.2.28'
%pip install promptflow-rag==0.1.0

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from pathlib import Path
from azure.ai.ml import Input, Output, command, load_component
from azure.ai.ml.entities import BuildContext, Environment
from azure.ai.ml.constants import AssetTypes, InputOutputModes
from azure.ai.ml.parallel import parallel_run_function, RunFunction

In [3]:
%%writefile config.json
{
    "subscription_id": "f464793a-e174-43cd-b473-47ac97c91075",
    "resource_group": "Helen-ML",
    "workspace_name": "ML-practice1"
}

Overwriting config.json


In [4]:
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient

identity = None
try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

ml_client = MLClient.from_config(credential=credential)

Found the config file in: ./config.json


In [5]:
from pathlib import Path
from azure.ai.ml.entities import BuildContext, Environment

"""
llm_rag_embeddings_doc_intel_environment = Environment(
    name="llm_rag_embeddings_doc_intel",
    description="AzureML RAGs base crack_and_chunk environment with azure-ai-formrecognizer installed.",
    build=BuildContext(path=Path.cwd() / "doc_intel_env"),
)
"""
llm_rag_embeddings_doc_intel_environment = ml_client.environments.get(
    name="llm_rag_embeddings_doc_intel", version="6"
)


Define the crack_and_chunk_with_doc_intel_component_parallel which can be used in place of the crack_and_chunk_parallel Component in Vector Index creation Pipelines.

Please reference this article for parallel job setup of ML pipeline. https://learn.microsoft.com/en-us/azure/machine-learning/how-to-use-parallel-job-in-pipeline?view=azureml-api-2&tabs=python 

Please reference this article for setting up optimum parameters of parallel job https://microsoft.github.io/azureml-ops-accelerator/4-Migrate/3-PerformanceTunePRS.html

In [6]:
crack_and_chunk_with_doc_intel_component_parallel = parallel_run_function(
    # version="0.0.1",
    name="crack_and_chunk_with_doc_intel_parallel",
    display_name="Crack and Chunk Data leveraging Azure AI Document Intelligence for PDFs in parallel",
    description="""Creates chunks from source data leveraging Azure AI Document Intelligence for PDFs in parallel.

    Supported formats: md, txt, html/htm, pdf, ppt(x), doc(x), xls(x), py""",
    inputs={
        # Input AzureML Data
        "input_data": Input(type="uri_folder", mode="ro_mount"),
        # Files to handle from source
        "input_glob": Input(
            type="string",
            default="/**/*",
            description="Limit files opened from `input_data`, defaults to '**/*'",
        ),
        "allowed_extensions": Input(
            type="string",
            optional=True,
            description="Comma separated list of extensions to include, if not provided the default list of supported extensions will be used. e.g. '.md,.txt,.html,.py,.pdf'",
        ),
        # Chunking options
        "chunk_size": Input(
            type="integer",
            default=768,
            description="Maximum number of tokens per chunk.",
        ),
        "chunk_overlap": Input(
            type="integer",
            default=0,
            description="Number of tokens to overlap between chunks.",
        ),
        "use_rcts": Input(
            type="boolean",
            default=True,
            description="Use langchain RecursiveTextSplitter to split chunks.",
        ),
        # Augmentation options
        "data_source_url": Input(
            type="string",
            optional=True,
            description="Base URL to join with file paths to create full source file URL for chunk metadata.",
        ),
        "document_path_replacement_regex": Input(
            type="string",
            optional=True,
            description="A JSON string with two fields, 'match_pattern' and 'replacement_pattern' to be used with re.sub on the source url. e.g. '{\"match_pattern\": \"(.*)/articles/(.*)\", \"replacement_pattern\": \"\\1/\\2\"}' would remove '/articles' from the middle of the url.",
        ),
        "doc_intel_connection_id": Input(
            type="string",
            description="AzureML Connection ID for Custom Workspace Connection containing the `endpoint` key and `api_key` secret for an Azure AI Document Intelligence Service.",
        ),
        "use_layout": Input(
            type="boolean",
            default=True,
            description="Use 'prebuilt-layout' model from Azure AI Document Intelligence, more expensive and slower but maintains more structure from original doc.",
        ),
    },
    outputs={
        "output_chunks": Output(type="uri_folder", mode="rw_mount"),
    },
    input_data="${{inputs.input_data}}",
    instance_count=4,
    max_concurrency_per_instance=4,
    mini_batch_size="1",
    mini_batch_error_threshold=-1,
    item_error_treshold=-1,
    retry_settings=dict(max_retries=2, timeout=1200),
    progress_update_timeout=259200,
    logging_level="DEBUG",
    task=RunFunction(
        code=Path.cwd() / "crack_and_chunk_with_doc_intel",
        entry_script="crack_and_chunk_parallel.py",
        program_arguments="--input_data ${{inputs.input_data}}\
        --input_glob '${{inputs.input_glob}}'\
        $[[--allowed_extensions ${{inputs.allowed_extensions}}]]\
        --output_chunks ${{outputs.output_chunks}}\
        --chunk_size ${{inputs.chunk_size}}\
        --chunk_overlap ${{inputs.chunk_overlap}}\
        --use_rcts ${{inputs.use_rcts}}\
        $[[--data_source_url ${{inputs.data_source_url}}]]\
        $[[--document_path_replacement_regex '${{inputs.document_path_replacement_regex}}']]\
        --doc_intel_connection_id '${{inputs.doc_intel_connection_id}}'\
        --use_layout ${{inputs.use_layout}}\ ",
        environment=llm_rag_embeddings_doc_intel_environment,
    ),
)

Define pipeline using the custom `crack_and_chunk_with_doc_intel_component_parallel along with the AzureML provided Components to embed and index your data.

In [7]:
ml_registry = MLClient(credential=ml_client._credential, registry_name="azureml")

# Reads input folder of files containing chunks and their metadata as batches, in parallel, and generates embeddings for each chunk. Output format is produced and loaded by `azureml.rag.embeddings.EmbeddingContainer`.
generate_embeddings_component = ml_registry.components.get(
    "llm_rag_generate_embeddings", label="latest"
)
# Reads an input folder produced by `azureml.rag.embeddings.EmbeddingsContainer.save()` and pushes all documents (chunk, metadata, embedding_vector) into an Azure Cognitive Search index. Writes an MLIndex yaml detailing the index and embeddings model information.
update_acs_index_component = ml_registry.components.get(
    "llm_rag_update_acs_index", label="latest"
)
# Takes a uri to a storage location where an MLIndex yaml is stored and registers it as an MLIndex Data asset in the AzureML Workspace.
register_mlindex_asset_component = ml_registry.components.get(
    "llm_rag_register_mlindex_asset", label="latest"
)

In [8]:
from azure.ai.ml import Input, Output
from azure.ai.ml.dsl import pipeline
from azure.ai.ml.entities._job.pipeline._io import PipelineInput

# Retrieve an already attached Azure Machine Learning Compute.
cpu_compute_target = "cpu-cluster2"


def use_automatic_compute(component):
    component.set_resources(
        instance_count=4,
        max_concurrency_per_instance=4,
        mini_batch_size=1,
        mini_batch_error_threshold=-1,
        retry_settings=dict(max_retries=2, timeout=1200),
        item_error_treshold=-1,
        progress_update_timeout=259200,
        properties={"compute_specification": {"automatic": True}},
        logging_level="DEBUG",
    )
    return component


def optional_pipeline_input_provided(input: PipelineInput):
    return input._data is not None


@pipeline(default_compute=cpu_compute_target)
def uri_to_acs_parallel(
    input_data: Input,
    doc_intel_connection_id: str,
    embeddings_model: str,
    acs_config: str,
    acs_connection_id: str,
    asset_name: str,
    chunk_size: int = 1024,
    data_source_glob: str = "*pdf",
    data_source_url: str = None,
    document_path_replacement_regex: str = None,
    use_layout: bool = True,
    aoai_connection_id: str = None,
    embeddings_container: Input = None,
):
    crack_and_chunk = crack_and_chunk_with_doc_intel_component_parallel(
        input_data=input_data,
        input_glob=data_source_glob,
        chunk_size=chunk_size,
        use_rcts=True,
        data_source_url=data_source_url,
        document_path_replacement_regex=document_path_replacement_regex,
        doc_intel_connection_id=doc_intel_connection_id,
        use_layout=True,
    )

    crack_and_chunk.inputs.input_data.mode = InputOutputModes.DOWNLOAD

    generate_embeddings = generate_embeddings_component(
        chunks_source=crack_and_chunk.outputs.output_chunks,
        embeddings_container=embeddings_container,
        embeddings_model=embeddings_model,
    )

    use_automatic_compute(generate_embeddings)

    if optional_pipeline_input_provided(aoai_connection_id):
        generate_embeddings.environment_variables[
            "AZUREML_WORKSPACE_CONNECTION_ID_AOAI"
        ] = aoai_connection_id
    if optional_pipeline_input_provided(embeddings_container):
        # If provided, previous_embeddings is expected to be a URI to an 'embeddings container' folder.
        # Each folder under this folder is generated by a `generate_embeddings_component` run and can be reused for subsequent embeddings runs.
        generate_embeddings.outputs.embeddings = Output(
            type="uri_folder", path=f"{embeddings_container.path}/{{name}}"
        )

    update_acs_index = update_acs_index_component(
        embeddings=generate_embeddings.outputs.embeddings,
        acs_config=acs_config,
    )
    if optional_pipeline_input_provided(acs_connection_id):
        update_acs_index.environment_variables[
            "AZUREML_WORKSPACE_CONNECTION_ID_ACS"
        ] = acs_connection_id

    register_mlindex = register_mlindex_asset_component(
        storage_uri=update_acs_index.outputs.index,
        asset_name=asset_name,
    )
    return {
        "mlindex_asset_uri": update_acs_index.outputs.index,
        "mlindex_asset_id": register_mlindex.outputs.asset_id,
    }

Get the connections to Azure OpenAI (for embeddings with `text-embedding-ada-002`) and Azure Cognitive Search.

In [9]:
aoai_connection = ml_client.connections.get("AOAI-westus")
acs_connection = ml_client.connections.get("cog-serch-westus")

Method connections: This is an experimental method, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


Create a Custom Connection with details for an Azure AI Document Intelligence Service.
[Setup instructions for Azure AI Document Intelligence](https://learn.microsoft.com/azure/ai-services/document-intelligence/create-document-intelligence-resource?view=doc-intel-3.1.0)

Use the Connections UI in an AzureML Workspace, under the Promptflow tab, to create a connection with these fields: ![custom_doc_intel_connection.png](./assets/custom_doc_intel_connection.png)

It's not yet supported to create/retrieve Custom Connections using SDK, so you will need to create it using the UI and we'll use string replacement below to get the ID for this custom connection to pass to our pipeline.

In [10]:
document_intelligence_connection_id = aoai_connection.id.replace(
    "AOAI-westus", "doc-intelligence"
)
document_intelligence_connection_id

'/subscriptions/f464793a-e174-43cd-b473-47ac97c91075/resourceGroups/Helen-ML/providers/Microsoft.MachineLearningServices/workspaces/ML-practice1/connections/doc-intelligence'

In [11]:
import json
from azure.ai.ml import Input

embeddings_model = (
    "azure_open_ai://deployment/text-embedding-ada-002/model/text-embedding-ada-002"
)

data_source = "wasbs://doc-intelligence-container9@helenworkspace6069790955.blob.core.windows.net"
# data_source = Path.cwd() / "pdfs"
asset_name = f"my-index-simple"

pipeline_job = uri_to_acs_parallel(
    input_data=Input(type="uri_folder", path=str(data_source)),
    # data_source_glob="**/*",
    data_source_glob="*pdf",
    data_source_url=None,
    document_path_replacement_regex=None,
    doc_intel_connection_id=document_intelligence_connection_id,
    use_layout=True,
    embeddings_model=embeddings_model,
    aoai_connection_id=aoai_connection.id,
    embeddings_container=Input(
        type="uri_folder",
        path=f"azureml://datastores/workspaceblobstore/paths/embeddings/{asset_name}",
    ),
    acs_config=json.dumps(
        {
            "index_name": asset_name,
        }
    ),
    acs_connection_id=acs_connection.id,
    asset_name=asset_name,
)
pipeline_job.display_name = asset_name

# Properties for Vector Index UI
pipeline_job.properties["azureml.mlIndexAssetName"] = asset_name
pipeline_job.properties["azureml.mlIndexAssetKind"] = "acs"
pipeline_job.properties["azureml.mlIndexAssetSource"] = "AzureML Data"

In [12]:
submission_name = f"crack-parallel-embedding-serial"
print(f"Submitting pipeline job to experiment: {submission_name}")
running_pipeline_job = ml_client.jobs.create_or_update(
    pipeline_job, experiment_name=submission_name
)

print(f"Submitted run, url: {running_pipeline_job.studio_url}")

Class AutoDeleteSettingSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class AutoDeleteConditionSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class BaseAutoDeleteSettingSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class IntellectualPropertySchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class ProtectionLevelSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class BaseIntellectualPropertySchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
pathOnCompute is not a known attribute

Submitting pipeline job to experiment: crack-parallel-embedding-serial
Submitted run, url: https://ml.azure.com/runs/goofy_camel_rmfv43qfht?wsid=/subscriptions/f464793a-e174-43cd-b473-47ac97c91075/resourcegroups/Helen-ML/workspaces/ML-practice1&tid=16b3c013-d300-468d-ac64-7eda0820b6d3


In [13]:
ml_client.jobs.stream(running_pipeline_job.name)

RunId: goofy_camel_rmfv43qfht
Web View: https://ml.azure.com/runs/goofy_camel_rmfv43qfht?wsid=/subscriptions/f464793a-e174-43cd-b473-47ac97c91075/resourcegroups/Helen-ML/workspaces/ML-practice1

Streaming logs/azureml/executionlogs.txt

[2024-05-11 03:09:08Z] Completing processing run id 3168ea0b-7e00-4a7f-9a6c-5867d15c1080.
[2024-05-11 03:09:09Z] Completing processing run id 092b6992-17cc-47bc-8baf-e916b26d0ff6.
[2024-05-11 03:09:10Z] Completing processing run id 47fc3b76-ab91-4d10-9bcc-9b3f147b5e04.
[2024-05-11 03:09:11Z] Completing processing run id f1e3d088-f22b-4a3d-924c-98db2567cd83.
[2024-05-11 03:09:12Z] Finishing experiment: no runs left and nothing to schedule.

Execution Summary
RunId: goofy_camel_rmfv43qfht
Web View: https://ml.azure.com/runs/goofy_camel_rmfv43qfht?wsid=/subscriptions/f464793a-e174-43cd-b473-47ac97c91075/resourcegroups/Helen-ML/workspaces/ML-practice1



In [14]:
%pip install azure-search-documents==11.4.0 b8

Note: you may need to restart the kernel to use updated packages.


In [20]:
from promptflow.rag import get_langchain_retriever_from_index

question = "What are the employees benefits"

my_index=ml_client.indexes.get(name=asset_name, label="latest")
index_langchain_retriever=get_langchain_retriever_from_index("azureml:/workspaces/645ce2a7-40bf-4da3-880b-ecb73f06fbb0/datastores/workspaceblobstore/paths/azureml/e458bd45-8ad6-49c8-b50e-bc329e55a101/index/")
index_langchain_retriever.get_relevant_documents(question)

FileNotFoundError: [Errno 2] No such file or directory: 'azureml:/workspaces/645ce2a7-40bf-4da3-880b-ecb73f06fbb0/datastores/workspaceblobstore/paths/azureml/e458bd45-8ad6-49c8-b50e-bc329e55a101/index/MLIndex'

In [17]:
print(my_index)

creation_context:
  created_at: '2024-05-11T02:45:25.796479+00:00'
  created_by: Helen Zeng
  created_by_type: User
  last_modified_at: '2024-05-11T02:45:25.809769+00:00'
id: azureml://locations/westus/workspaces/645ce2a7-40bf-4da3-880b-ecb73f06fbb0/data/my-index-simple/versions/1
name: my-index-simple
path: azureml://workspaces/645ce2a7-40bf-4da3-880b-ecb73f06fbb0/datastores/workspaceblobstore/paths/azureml/e458bd45-8ad6-49c8-b50e-bc329e55a101/index/
properties:
  azureml.mlIndexAsset: 'true'
  azureml.mlIndexAssetKind: acs
  azureml.mlIndexAssetPipelineRunId: hungry_dolphin_vvfpthj4fr
  azureml.mlIndexAssetSource: AzureML Data
stage: Development
tags:
  PlaygroundType: '[''Text'']'
version: '1'

